In [1]:
#main
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
from pandas.plotting import scatter_matrix
import pandas_profiling

In [2]:
# support
# csv to pd loader
import corelib.data_loader as data_loader

# reduce memory usage
import corelib.data_prep as data_prep

# dump data with
import corelib.data_dump as data_dump

# saving images
import corelib.data_vis as data_vis
# for ignoring SciPy warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [3]:
from imp import reload

In [ ]:
reload(data_loader)

In [ ]:
reload(data_prep)

In [ ]:
reload(data_dump)

In [ ]:
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from sklearn.preprocessing import FunctionTransformer

from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
#encoders
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.backward_difference import BackwardDifferenceEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.hashing import HashingEncoder
from category_encoders.binary import BinaryEncoder
from category_encoders.polynomial import PolynomialEncoder

# Preprocessing

### dump from this point

In [4]:
print(data_loader.loader.__doc__)


    Unpack data, view .csv files or return pandas dataframe
    
    Parameters
    ----------
    :param mode:
    Mode of function. 
    Available:
    'tree' show  available files
    'view' show first five strings of data files
    'extract' extract data into dict of objects
        string

    :param path:
    Current path to folder with data
        string, default DATA_PATH

    :param data_for_load:
    Dictionary where keys are file names and values are dicts of parameters.
    Looks like {'this.csv': {'sep': ',', 'encoding': 'utf-8'}} etc.
    If None all files are loaded wit default parameters.
        dict, default None

    Parameters are available:

    :sep:
    Delimiter to use
        string, default ','

    :index_col:
    Column to use as the row labels of the DataFrame, either given as string name or column index.
    If a sequence of int/str is given, a MultiIndex is used.
    Note: index_col=False can be used to force pandas to not use the first column as the in

In [5]:
data_loader.loader(mode='tree')

anydata.csv ..... C:\xampp\htdocs\_uroki_\python-learning\ml_support\input\notempty\anydata.csv
most_deeped.txt ..... C:\xampp\htdocs\_uroki_\python-learning\ml_support\input\notempty\notempty_deeper\most_deeped.txt
sample.csv ..... C:\xampp\htdocs\_uroki_\python-learning\ml_support\input\sample.csv
samplezip.zip ..... C:\xampp\htdocs\_uroki_\python-learning\ml_support\input\samplezip.zip


In [6]:
data_loader.loader(mode='view')

First five lines of viewed files:

anydata.csv
-----
installation_id,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world
0001e90f,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
0001e90f,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
0001e90f,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0,""event_code"":2000}",1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
0001e90f,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!"",""identifier"":""Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape"",""media_type"":""audio"",""total_duration"":6758,""event_count"":2,""ga

In [7]:
data = data_loader.loader(mode='extract')

List of extracted files:
anydata.csv
sample.csv
samplezip.zip


In [8]:
data_seq1 = data_loader.loader(mode='extract', data_for_load={'sample.csv':{}})

List of extracted files:
sample.csv


In [9]:
data_for_load = {'samplezip.zip': 
                 {'index_col': 'family_id', 
                  'dtype': {'assigned_day': np.float64}},
                'anydata.csv':
                {'parse_dates': ['timestamp']}}
data_seq2 = data_loader.loader(mode='extract', data_for_load=data_for_load)

List of extracted files:
samplezip.zip
anydata.csv


In [10]:
data_seq2['anydata.csv'].head(1)

,installation_id,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world
0,0001e90f,27253bdc,45bb1e1b6b50c07b,2019-09-06 17:53:46.937,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0,Welcome to Lost Lagoon!,Clip,NONE


In [11]:
df_train, df_test = data_seq2.values()

In [12]:
del data, data_seq1, data_seq2

In [13]:
# reduce memory usage
df_train = data_prep.reduce_mem_usage(df_train)
df_test = data_prep.reduce_mem_usage(df_test)

Mem. usage decreased to  0.00 Mb (37.5% reduction)
Mem. usage decreased to 13.73 Mb (18.2% reduction)


In [14]:
data = df_train, df_test

In [15]:
# Dump loaded and prepared data
data_dump.dumper(dump_list=data, path='loaded_data', method='shelve', task='s')

Object 0 is dumped to "loaded_data" objects
Object 1 is dumped to "loaded_data" objects


### to this point

In [16]:
# Dump open (prepared dataset)
df_train, df_test = data_dump.dumper(path='loaded_data', method='shelve', task='o')

# Visualisation

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.hist(bins=50, figsize=(10,6))
data_vis.save_fig("df_train_histogram_plots")
plt.show()

In [ ]:
# set an features from data to plot scatter_matrix
attributes = []
scatter_matrix(df_train[attributes], figsize=(12, 8))
data_vis.save_fig("df_train_correlation_plot")

In [ ]:
pandas_profiling.ProfileReport(df_train)

# Modeling

In [ ]:
#select the y

In [ ]:
# preprocessing pipline
pipePre = Pipeline([
    ('standardscaler', StandardScaler()),
    ('normalizer', Normalizer())
     ])

In [ ]:
df_train = pipePre.fit_transform(df_train)
df_test = pipePre.fit_transform(df_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size = 0.25, random_state=42)
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print(N_train, N_test)

In [ ]:
del df_train
del df_test